# Part A: Training from scratch



![CNN From Scrach](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/12/Screenshot-from-2018-12-10-15-09-20.png)



### Importing required libraries

In [ ]:
!pip install wandb

In [ ]:
import torch
import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, ConcatDataset, random_split
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
!pip install wandb
import wandb
os.environ["WANDB_NOTEBOOK_NAME"] = "PartA.ipynb"
wandb.login()
wandb.login(key='13718865007c1068066166d683d0ce0cf87ec304')

wandb: Currently logged in as: ge23m018. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ge22m009/.netrc


True

## Function to find mean and standard deviation of DataLoader

**1. For the GeeksforGeeks article:**

GeeksforGeeks. (n.d.). Computing the mean and std of a dataset in PyTorch. *GeeksforGeeks*. Retrieved from [https://www.geeksforgeeks.org/computing-the-mean-and-std-of-a-dataset-in-pytorch/](https://www.geeksforgeeks.org/computing-the-mean-and-std-of-a-dataset-in-pytorch/)

**2. For the PyTorch discussion thread:**

PyTorch Forums. (2018, November 20). Computing the mean and std of dataset. *PyTorch Discussion Forums*. Retrieved from [https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949/32](https://discuss.pytorch.org/t/computing-the-mean-and-std-of-dataset/34949/32)


In [ ]:
def get_mean_and_std(dataLoader):
    # Initialize variables to store mean and standard deviation
    mean = 0
    std = 0
    # Initialize total image count
    total_image_count = 0
    # Loop through the data loader
    for i, (images, labels) in enumerate(dataLoader):
        # Get the batch size
        batch_image_count = images.size(0)
        # Reshape the images
        images = images.view(batch_image_count, images.size(1), -1)
        # Calculate mean and standard deviation along the specified dimensions
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
        # Update total image count
        total_image_count += batch_image_count
    # Calculate mean and standard deviation across all images
    mean /= total_image_count
    std /= total_image_count
    # Return mean and standard deviation
    return mean, std


## Loading the iNatuaralist Dataset


### References for data transformations in PyTorch:

1. **PyTorch Documentation**:
   - Reference: [https://pytorch.org/docs/stable/torchvision/transforms.html](https://pytorch.org/docs/stable/torchvision/transforms.html)


2. **PyTorch Forums**:
   - Reference: [https://discuss.pytorch.org/t/composing-custom-transforms/6955](https://discuss.pytorch.org/t/composing-custom-transforms/6955)




In [ ]:
def load_dataset(data_augmentation, train_path, test_path, train_batch_size, val_batch_size, test_batch_size):
    """
    Function to load and preprocess datasets.

    Args:
    - data_augmentation (bool): Flag indicating whether to apply data augmentation.
    - train_path (str): Path to the directory containing the training dataset.
    - test_path (str): Path to the directory containing the test dataset.
    - train_batch_size (int): Batch size for training DataLoader.
    - val_batch_size (int): Batch size for validation DataLoader.
    - test_batch_size (int): Batch size for test DataLoader.

    Returns:
    - train_Loader (DataLoader): DataLoader for the training dataset.
    - val_Loader (DataLoader): DataLoader for the validation dataset.
    - test_Loader (DataLoader): DataLoader for the test dataset.
    """
    # Define transformation for training data
    transformer1 = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize images to 256x256
        transforms.ToTensor(),  # Convert images to PyTorch tensors
        transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])  # Normalize images
    ])

    # Load training dataset
    train_Dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transformer1)
    # Split training dataset into train and validation sets
    train_datasize = int(0.8 * len(train_Dataset))
    train_Dataset, val_Dataset = random_split(train_Dataset, [train_datasize, len(train_Dataset) - train_datasize])

    # Augment data if data_augmentation is True
    if data_augmentation:
        # Define data augmentation transformation
        transformer2 = transforms.Compose([
            transforms.Resize((256, 256)),  # Resize images to 256x256
            transforms.RandomHorizontalFlip(0.5),  # Random horizontal flip with probability 0.5
            transforms.RandomVerticalFlip(0.02),  # Random vertical flip with probability 0.02
            transforms.RandomRotation(degrees=45),  # Random rotation by maximum 45 degrees
            transforms.ToTensor(),  # Convert images to PyTorch tensors
            transforms.Normalize(mean=[0.4602, 0.4495, 0.3800], std=[0.2040, 0.1984, 0.1921])  # Normalize images
        ])
        # Load augmented dataset
        augmented_dataset = torchvision.datasets.ImageFolder(root=train_path, transform=transformer2)
        # Split augmented dataset
        augmented_dataset_size = int(0.2 * len(augmented_dataset))
        augmented_dataset, _ = random_split(augmented_dataset, [augmented_dataset_size, len(augmented_dataset) - augmented_dataset_size])
        # Concatenate original training dataset and augmented dataset
        train_Dataset = ConcatDataset([train_Dataset, augmented_dataset])

    # Create data loaders for training, validation, and test sets
    train_Loader = DataLoader(
        train_Dataset,
        batch_size=train_batch_size,
        shuffle=True)
    test_Loader = DataLoader(
        test_path,
        batch_size=test_batch_size,
        shuffle=True)
    val_Loader = DataLoader(
        val_Dataset,
        batch_size=val_batch_size,
        shuffle=True)

    return train_Loader, val_Loader, test_Loader


#### Mean = [0.4602, 0.4495, 0.3800] , std = [0.2040, 0.1984, 0.1921] for images of size 256 x 256

# Storing the Name of the Labels

In [ ]:
def defineClasses(train_path):
    classes = []
    for root,dirs, files in os.walk(train_path):
    #     for _file in files:
    #         print(_file)
        for _dir in dirs:
            classes.append(_dir)
    classes.sort()
    return classes

## Setting device to cuda if available


In [ ]:
# Check if CUDA is available, set device accordingly
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Create a CNN Model

![CNN From Scrach](https://docs.ecognition.com/Resources/Images/ECogUsr/UG_CNN_scheme.png)


In [ ]:
class create_lenet(torch.nn.Module):
    def __init__(self, num_classes, Kernel_size, num_filters,
                 activation_func, filter_factor, is_data_augment, dropout_factor):
        super(create_lenet, self).__init__()

        # Initialize activation functions
        self.actfunc1 = None
        self.actfunc2 = None
        self.actfunc3 = None
        self.actfunc4 = None
        self.actfunc5 = None

        # Define convolutional layers
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=num_filters, kernel_size=Kernel_size)
        self.conv2 = torch.nn.Conv2d(in_channels=num_filters, out_channels=int(num_filters * filter_factor), kernel_size=Kernel_size)
        self.conv3 = torch.nn.Conv2d(in_channels=int(num_filters * filter_factor), out_channels=int(num_filters * filter_factor ** 2), kernel_size=Kernel_size)
        self.conv4 = torch.nn.Conv2d(in_channels=int(num_filters * filter_factor ** 2), out_channels=int(num_filters * filter_factor ** 3), kernel_size=Kernel_size)
        self.conv5 = torch.nn.Conv2d(in_channels=int(num_filters * filter_factor ** 3), out_channels=int(num_filters * filter_factor ** 4), kernel_size=Kernel_size)

        # Calculate size after convolution and pooling operations
        self.size = (256 - Kernel_size) // 2 ** 5

        # Define maxpooling layers
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=Kernel_size, stride=2)
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=Kernel_size, stride=2)
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=Kernel_size, stride=2)
        self.maxpool4 = torch.nn.MaxPool2d(kernel_size=Kernel_size, stride=2)
        self.maxpool5 = torch.nn.MaxPool2d(kernel_size=Kernel_size, stride=2)

        # Set activation functions based on input argument
        if activation_func == 'ReLU':
            self.actfunc1 = torch.nn.ReLU()
            self.actfunc2 = torch.nn.ReLU()
            self.actfunc3 = torch.nn.ReLU()
            self.actfunc4 = torch.nn.ReLU()
            self.actfunc5 = torch.nn.ReLU()
        elif activation_func == 'GELU':
            self.actfunc1 = torch.nn.GELU()
            self.actfunc2 = torch.nn.GELU()
            self.actfunc3 = torch.nn.GELU()
            self.actfunc4 = torch.nn.GELU()
            self.actfunc5 = torch.nn.GELU()
        elif activation_func == 'SiLU':
            self.actfunc1 = torch.nn.SiLU()
            self.actfunc2 = torch.nn.SiLU()
            self.actfunc3 = torch.nn.SiLU()
            self.actfunc4 = torch.nn.SiLU()
            self.actfunc5 = torch.nn.SiLU()
        elif activation_func == 'Mish':
            self.actfunc1 = torch.nn.Mish()
            self.actfunc2 = torch.nn.Mish()
            self.actfunc3 = torch.nn.Mish()
            self.actfunc4 = torch.nn.Mish()
            self.actfunc5 = torch.nn.Mish()
        else:
            # Default activation function is ReLU
            self.actfunc1 = torch.nn.ReLU()
            self.actfunc2 = torch.nn.ReLU()
            self.actfunc3 = torch.nn.ReLU()
            self.actfunc4 = torch.nn.ReLU()
            self.actfunc5 = torch.nn.ReLU()

        # Define dropout layers
        self.dropp1 = torch.nn.Dropout(dropout_factor)
        self.dropp2 = torch.nn.Dropout(dropout_factor)

        # Define fully connected layers
        self.fc1 = torch.nn.Linear(in_features=self.size ** 2 * int(num_filters * filter_factor ** 4), out_features=self.size ** 2 * int(num_filters * filter_factor ** 4))
        self.fc2 = torch.nn.Linear(in_features=self.size ** 2 * int(num_filters * filter_factor ** 4), out_features=num_classes)

        # LogSoftmax for output
        self.logSoftmax = torch.nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Forward pass through convolutional layers and activation functions
        x = self.conv1(x)
        x = self.actfunc1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.actfunc2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.actfunc3(x)
        x = self.maxpool3(x)

        x = self.conv4(x)
        x = self.actfunc4(x)
        x = self.maxpool4(x)

        x = self.conv5(x)
        x = self.actfunc5(x)
        x = self.maxpool5(x)

        # Reshape for fully connected layers
        x = x.view(-1, self.size ** 2 * int(num_filters * filter_factor ** 4))

        # Forward pass through fully connected layers and dropout layers
        x = self.fc1(x)
        x = self.dropp1(x)
        x = self.fc2(x)
        x = self.dropp2(x)

        # Output
        output = self.logSoftmax(x)
        return output


# Create CNN Model

### Counting the size of train and test dataset

In [ ]:
def get_train_test_count(train_pat, test_pat):
    train_count = len(glob.glob(train_path+'/**/*.jpg'))
    test_count = len(glob.glob(test_path+'/**/*.jpg'))
    print("Training dataset count : ", train_count)
    print("Validation dataset count", test_count)
    return train_count, test_count

# Training the model

In [ ]:
def train(cnn, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log):
    """
    Function to train a convolutional neural network.

    Args:
    - cnn (torch.nn.Module): The convolutional neural network model.
    - learning_rate (float): Learning rate for optimization.
    - epochs (int): Number of epochs for training.
    - train_Loader (DataLoader): DataLoader for training dataset.
    - val_Loader (DataLoader): DataLoader for validation dataset.
    - train_count (int): Total number of samples in the training dataset.
    - test_count (int): Total number of samples in the validation dataset.
    - is_wandb_log (bool): Flag indicating whether to log metrics using Weights & Biases.

    Returns:
    None
    """
    # Define loss function and optimizer
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=cnn.parameters(), lr=learning_rate, weight_decay=1e-4)

    # Iterate over epochs
    for epoch in range(epochs):
        # Initialize training accuracy and loss for each epoch
        train_accuracy = 0
        train_loss = 0
        cnn.train()  # Set the model to training mode
        # Iterate over batches in the training DataLoader
        for i, (images, labels) in enumerate(train_Loader):
            # Move images and labels to device (GPU if available)
            images, labels = images.to(device), labels.to(device)
            # Zero the gradients
            optimizer.zero_grad()
            # Forward propagation
            y_pred = cnn(images)
            # Calculate loss
            loss = loss_function(y_pred, labels)
            # Backward propagation
            loss.backward()
            # Update weights
            optimizer.step()
            # Accumulate training loss
            train_loss += loss.item()
            # Calculate training accuracy
            _, prediction = torch.max(y_pred.data, 1)
            train_accuracy += int(torch.sum(prediction == labels.data))

        # Compute average training accuracy and loss for the epoch
        train_accuracy /= train_count
        train_loss /= train_count
        # Print training accuracy and loss for the epoch
        print(f"Epochs : {epoch + 1} Train Accuracy : {train_accuracy} Train Loss {train_loss}")

        # Initialize validation accuracy and loss for each epoch
        val_accuracy = 0
        val_loss = 0
        # Disable gradient computation for validation
        with torch.no_grad():
            cnn.eval()  # Set the model to evaluation mode
            # Iterate over batches in the validation DataLoader
            for i, (images, labels) in enumerate(val_Loader):
                # Move images and labels to device (GPU if available)
                images, labels = images.to(device), labels.to(device)
                # Forward propagation
                y_pred = cnn(images)
                # Calculate loss
                loss = loss_function(y_pred, labels)
                # Accumulate validation loss
                val_loss += loss.item()
                # Calculate validation accuracy
                _, predicted = torch.max(y_pred.data, 1)
                val_accuracy += int(torch.sum(predicted == labels.data))

            # Compute average validation accuracy and loss for the epoch
            val_accuracy /= test_count
            val_loss /= test_count

            # Print validation accuracy and loss for the epoch
            print(f"Epochs : {epoch + 1} Validation Accuracy : {val_accuracy} Validation Loss {val_loss}")
            # Log metrics using Weights & Biases if enabled
            if is_wandb_log:
                wandb.log({"train_accuracy": train_accuracy, "train_loss": train_loss, "val_accuracy": val_accuracy,
                           "val_loss": val_loss})


## Function to get the Test Accuracy & Test Loss

In [ ]:
def test(cnn, num_classes, test_path, test_batch_size , num_filters, kernel_size,
         activation_func, filter_factor, learning_rate, epochs, dropout_factor):
    """
    Function to test the trained CNN model on the test dataset.

    Args:
    - cnn (torch.nn.Module): The trained convolutional neural network model.
    - num_classes (int): Number of classes in the dataset.
    - test_path (str): Path to the test dataset.
    - test_batch_size (int): Batch size for testing.
    - num_filters (int): Number of filters in the convolutional layers.
    - kernel_size (int): Size of the convolutional kernel.
    - activation_func (str): Activation function to be used in the model.
    - filter_factor (float): Factor to determine the number of filters in subsequent layers.
    - learning_rate (float): Learning rate for optimization.
    - epochs (int): Number of epochs for testing.
    - dropout_factor (float): Dropout factor for regularization.

    Returns:
    None
    """
    # Define transformation for test data
    transformer = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])

    # Load test dataset
    test_Dataset = torchvision.datasets.ImageFolder(root=test_path, transform=transformer)
    # Create DataLoader for test dataset
    test_Loader = DataLoader(
        test_Dataset,
        batch_size=test_batch_size,
        shuffle=True)
    # Count the number of test samples
    test_count = len(glob.glob(test_path+'/**/*.jpg'))
    print('Testing the Model...')
    # Define loss function and optimizer
    loss_function = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params=cnn.parameters(), lr=learning_rate)

    # Iterate over epochs
    for epoch in range(epochs):
        test_accuracy = 0
        test_loss = 0
        # Disable gradient computation for validation
        with torch.no_grad():
            cnn.eval()  # Set the model to evaluation mode
            # Iterate over batches in the test DataLoader
            for i, (images, labels) in enumerate(test_Loader):
                # Move images and labels to device (GPU if available)
                images, labels = images.to(device), labels.to(device)
                # Forward propagation
                y_pred = cnn(images)
                # Calculate loss
                loss = loss_function(y_pred, labels)
                # Accumulate test loss
                test_loss += loss.item()
                # Calculate test accuracy
                _, predicted = torch.max(y_pred.data, 1)
                test_accuracy += int(torch.sum(predicted == labels.data))

            # Compute average test accuracy and loss for the epoch
            test_accuracy /= test_count
            test_loss /= test_count

            # Print test accuracy and loss for the epoch
            print(f"Epochs : {epoch + 1} Test Accuracy : {test_accuracy} Test Loss {test_loss}")


In [ ]:
def main(num_classes, kernel_size,  train_path, test_path, train_batch_size, val_batch_size, test_batch_size, num_filters,
         activation_func, filter_factor, is_data_augment, learning_rate, epochs, is_wandb_log, dropout_factor):
    print("train_path", train_path)
    print("test_path", test_path)
    train_Loader, val_Loader, test_Loader = load_dataset(is_data_augment, train_path, test_path, train_batch_size, val_batch_size, test_batch_size)
    cnn = create_lenet(num_classes, kernel_size, num_filters, activation_func, filter_factor, is_data_augment, dropout_factor)
    cnn = cnn.to(device)
    train_count, test_count = get_train_test_count(train_path, test_path)
    print("Training the Model...")
    train(cnn, learning_rate, epochs, train_Loader, val_Loader, train_count, test_count, is_wandb_log)
    print("Training Finished !!")
    return cnn



## Running Sweep

In [ ]:
import wandb
train_path = '/home/ge22m009/inaturalist_12K/train/'
test_path = '/home/ge22m009/inaturalist_12K/val/'
train_batch_size = 64
test_batch_size = 16
val_batch_size = 16
num_classes = 10
kernel_size = 3
is_wandb_log = True
is_data_augment=True


In [ ]:
def run_sweep():

    default_params =dict(
        num_filters = 32,
        activation_func = 'ReLU',
        filter_factor = 2,
        learning_rate = 0.001,
        epochs = 3,
        dropout_factor = 0,
    )

    run = wandb.init(config=default_params, project='dl_ge23m18')

    config = wandb.config
    num_filters = config.num_filters
    activation_func = config.activation_func
    filter_factor = config.filter_factor
    learning_rate = config.learning_rate
    epochs = config.epochs
    dropout_factor = config.dropout_factor

    run.name = 'ac_' + activation_func + '_nf_' + str(num_filters) + '_ff_'+ str(filter_factor)+'_df_'+str(dropout_factor)
    main(num_classes, kernel_size,  train_path, test_path, train_batch_size, val_batch_size, test_batch_size, num_filters,
         activation_func, filter_factor, is_data_augment, learning_rate, epochs, is_wandb_log, dropout_factor)



In [ ]:
#Set up a sweep config
sweep_config = {
    'metric': {
        'goal': 'maximize',
        'name': 'val_accuracy'
        },
    "method": "bayes",
    "project": "dl_ge23m18",
    "parameters": {
        "num_filters": {
            "values": [32, 64]
        },
        "filter_factor": {
            "values": [0.5]
        },
        "epochs": {
            "values": [5, 10]
        },
        "activation_func":
        {
            "values" : ["ReLU", "GELU", "SiLU", "Mish"]
        },
        "dropout_factor":
        {
            "values": [0 , 0.3, 0.4]
        },
    }
}

# creating the sweep
sweep_id = wandb.sweep(sweep_config, project="dl_ge23m18")

Create sweep with ID: m9wnq4zl
Sweep URL: https://wandb.ai/ge23m018/dl_ge23m18/sweeps/m9wnq4zl


In [ ]:
wandb.agent(sweep_id, function=run_sweep)

wandb: Agent Starting Run: b3rjda62 with config:
wandb: 	activation_func: GELU
wandb: 	dropout_factor: 0.4
wandb: 	epochs: 5
wandb: 	filter_factor: 0.5
wandb: 	num_filters: 32
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_path /home/ge22m009/inaturalist_12K/train/
test_path /home/ge22m009/inaturalist_12K/val/
Training dataset count :  9999
Validation dataset count 2000
Training the Model...
Epochs : 1 Train Accuracy : 0.11751175117511752 Train Loss 0.03606993356386725
Epochs : 1 Validation Accuracy : 0.16 Validation Loss 0.14013563656806946
Epochs : 2 Train Accuracy : 0.16051605160516053 Train Loss 0.03517289516484455
Epochs : 2 Validation Accuracy : 0.212 Validation Loss 0.13597732031345366
Epochs : 3 Train Accuracy : 0.17551755175517553 Train Loss 0.03474460385873182
Epochs : 3 Validation Accuracy : 0.2175 Validation Loss 0.13562490910291672
Epochs : 4 Train Accuracy : 0.1858185818581858 Train Loss 0.034535030243766106
Epochs : 4 Validation Accuracy : 0.218 Validation Loss 0.13485996156930924
Epochs : 5 Train Accuracy : 0.18351835183518353 Train Loss 0.03447690018654728
Epochs : 5 Validation Accuracy : 0.229 Validation Loss 0.13458968657255171
Training Finished !!


train_accuracy,▁▅▇██
train_loss,█▄▂▁▁
val_accuracy,▁▆▇▇█
val_error,█▃▂▁▁
train_accuracy,0.18352
train_loss,0.03448
val_accuracy,0.229
val_error,0.13459


wandb: Agent Starting Run: pyug4ak7 with config:
wandb: 	activation_func: ReLU
wandb: 	dropout_factor: 0
wandb: 	epochs: 10
wandb: 	filter_factor: 0.5
wandb: 	num_filters: 32
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_path /home/ge22m009/inaturalist_12K/train/
test_path /home/ge22m009/inaturalist_12K/val/
Training dataset count :  9999
Validation dataset count 2000
Training the Model...
Epochs : 1 Train Accuracy : 0.1542154215421542 Train Loss 0.035326327952352424
Epochs : 1 Validation Accuracy : 0.1725 Validation Loss 0.13719529336690903
Epochs : 2 Train Accuracy : 0.17941794179417941 Train Loss 0.034527491469277846
Epochs : 2 Validation Accuracy : 0.19 Validation Loss 0.13605467510223387
Epochs : 3 Train Accuracy : 0.20952095209520952 Train Loss 0.03400065620156071
Epochs : 3 Validation Accuracy : 0.2105 Validation Loss 0.13395568805932997
Epochs : 4 Train Accuracy : 0.22372237223722372 Train Loss 0.03324726901420153
Epochs : 4 Validation Accuracy : 0.2225 Validation Loss 0.13318588489294053
Epochs : 5 Train Accuracy : 0.24332433243324333 Train Loss 0.03279165554456752
Epochs : 5 Validation Accuracy : 0.222 Validation Loss 0.1321647279858589
Epochs : 6 Train Accuracy : 0.2532253225322532 Tra

...

train_accuracy,▁▄▆▆▇▇▇███
train_loss,█▅▄▃▃▂▂▁▁▁
val_accuracy,▁▃▄▅▆▇▆▇▆█
val_error,█▆▄▃▃▂▂▂▂▁
train_accuracy,0.35294
train_loss,0.0287
val_accuracy,0.34
val_error,0.11768


wandb: Agent Starting Run: q0abtas3 with config:
wandb: 	activation_func: Mish
wandb: 	dropout_factor: 0
wandb: 	epochs: 10
wandb: 	filter_factor: 0.5
wandb: 	num_filters: 64
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_path /home/ge22m009/inaturalist_12K/train/
test_path /home/ge22m009/inaturalist_12K/val/
Training dataset count :  9999
Validation dataset count 2000
Training the Model...
Epochs : 1 Train Accuracy : 0.17641764176417643 Train Loss 0.03462775053531125
Epochs : 1 Validation Accuracy : 0.2165 Validation Loss 0.1326683611869812
Epochs : 2 Train Accuracy : 0.2538253825382538 Train Loss 0.03229768495104744
Epochs : 2 Validation Accuracy : 0.2505 Validation Loss 0.12780801022052765
Epochs : 3 Train Accuracy : 0.2836283628362836 Train Loss 0.031388811569641156
Epochs : 3 Validation Accuracy : 0.291 Validation Loss 0.12483191096782684
Epochs : 4 Train Accuracy : 0.2936293629362936 Train Loss 0.03071286716703439
Epochs : 4 Validation Accuracy : 0.291 Validation Loss 0.12374093705415726
Epochs : 5 Train Accuracy : 0.3144314431443144 Train Loss 0.030161394156364336
Epochs : 5 Validation Accuracy : 0.314 Validation Loss 0.1211233177781105
Epochs : 6 Train Accuracy : 0.3219321932193219 Train L

train_accuracy,▁▄▅▅▆▆▇▇██
train_loss,█▅▄▄▃▂▂▂▁▁
val_accuracy,▁▃▅▅▇▇▆█▇█
val_error,█▆▄▄▃▂▃▁▁▁
train_accuracy,0.36474
train_loss,0.02851
val_accuracy,0.34
val_error,0.11722


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


## Run Without sweep

In [ ]:
train_path = '/home/ge22m009/inaturalist_12K/train/'
test_path = '/home/ge22m009/inaturalist_12K/val/'
train_batch_size = 64
test_batch_size = 16
val_batch_size = 16
num_classes = 10
kernel_size = 3
is_wandb_log = False
is_data_augment=True
num_filters = 32
activation_func = 'Mish'
filter_factor = 1
learning_rate = 0.001
epochs = 10
dropout_factor = 0.3

main(num_classes, kernel_size,  train_path, test_path, train_batch_size, val_batch_size, test_batch_size, num_filters,
     activation_func, filter_factor, is_data_augment, learning_rate, epochs, is_wandb_log, dropout_factor)


train_path /home/ge22m009/inaturalist_12K/train/
test_path /home/ge22m009/inaturalist_12K/val/
Training dataset count :  9999
Validation dataset count 2000
Training the Model...
Epochs : 1 Train Accuracy : 0.19041904190419043 Train Loss 0.034574953445566575
Epochs : 1 Validation Accuracy : 0.2735 Validation Loss 0.1282467455267906
Epochs : 2 Train Accuracy : 0.24972497249724973 Train Loss 0.032914484318571934
Epochs : 2 Validation Accuracy : 0.287 Validation Loss 0.12547618907690047
Epochs : 3 Train Accuracy : 0.2634263426342634 Train Loss 0.03225637914085522
Epochs : 3 Validation Accuracy : 0.3055 Validation Loss 0.12193633139133453
Epochs : 4 Train Accuracy : 0.28152815281528154 Train Loss 0.03163011890254577
Epochs : 4 Validation Accuracy : 0.325 Validation Loss 0.11930974096059799
Epochs : 5 Train Accuracy : 0.29262926292629265 Train Loss 0.03127662439753573
Epochs : 5 Validation Accuracy : 0.34 Validation Loss 0.11682473808526993
Epochs : 6 Train Accuracy : 0.30183018301830183 Tra

create_lenet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (actfunc1): Mish()
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (actfunc2): Mish()
  (maxpool2): MaxPool2d(kernel_size=3, stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (actfunc3): Mish()
  (maxpool3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (actfunc4): Mish()
  (maxpool4): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (actfunc5): Mish()
  (maxpool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=800, out_features=800, bias=True)
  (dropp1): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_feat

### Testing on the best hyperparameters

In [ ]:
test_path = '/home/ge22m009/inaturalist_12K/val/'
test_batch_size = 16
train_batch_size = 64
val_batch_size = 16
num_classes = 10
is_data_augment=True
is_wandb_log = False
kernel_size = 3
num_filters = 32
activation_func = 'Mish'
filter_factor = 1
learning_rate = 0.001
epochs = 5
dropout_factor = 0.3

# test(num_classes, test_path, test_batch_size , num_filters, kernel_size,  activation_func, filter_factor, learning_rate, epochs, dropout_factor)
cnn = main(num_classes, kernel_size,  train_path, test_path, train_batch_size, val_batch_size, test_batch_size, num_filters,
           activation_func, filter_factor, is_data_augment, learning_rate, epochs, is_wandb_log, dropout_factor)
test(cnn, num_classes, test_path, test_batch_size , num_filters, kernel_size,
     activation_func, filter_factor, learning_rate, epochs, dropout_factor)

train_path /home/ge22m009/inaturalist_12K/train/
test_path /home/ge22m009/inaturalist_12K/val/
Training dataset count :  9999
Validation dataset count 2000
Training the Model...
Epochs : 1 Train Accuracy : 0.19171917191719173 Train Loss 0.03443243233176849
Epochs : 1 Validation Accuracy : 0.269 Validation Loss 0.1300344308614731
Epochs : 2 Train Accuracy : 0.24862486248624863 Train Loss 0.032904511380760726
Epochs : 2 Validation Accuracy : 0.2755 Validation Loss 0.12559407448768617
Epochs : 3 Train Accuracy : 0.2777277727772777 Train Loss 0.03193540077875204
Epochs : 3 Validation Accuracy : 0.3165 Validation Loss 0.12300641524791718
Epochs : 4 Train Accuracy : 0.29492949294929494 Train Loss 0.03135473657362532
Epochs : 4 Validation Accuracy : 0.315 Validation Loss 0.12160956716537476
Epochs : 5 Train Accuracy : 0.28802880288028804 Train Loss 0.031148259026227636
Epochs : 5 Validation Accuracy : 0.3245 Validation Loss 0.11878633201122284
Training Finished !!
Testing the Model...
Epochs 

In [ ]:
test(cnn, num_classes, test_path, test_batch_size , num_filters, kernel_size,
     activation_func, filter_factor, learning_rate, epochs, dropout_factor)

Testing the Model...
Epochs : 1 Test Accuracy : 0.153 Test Loss 0.14407220548391342
Epochs : 2 Test Accuracy : 0.153 Test Loss 0.14407220566272735
Epochs : 3 Test Accuracy : 0.153 Test Loss 0.1440722049474716
Epochs : 4 Test Accuracy : 0.153 Test Loss 0.1440722049474716
Epochs : 5 Test Accuracy : 0.153 Test Loss 0.14407220506668092
